In [ ]:
# ✅ Step 1: Install AIQ Toolkit + LangChain plugin
!pip install -U pip
!pip install git+https://github.com/NVIDIA/NeMo-Agent-Toolkit.git
!pip install aiqtoolkit-langchain langchain-nvidia-ai-endpoints sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/NVIDIA/NeMo-Agent-Toolkit.git to /tmp/pip-req-build-62ki0reo
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo-Agent-Toolkit.git /tmp/pip-req-build-62ki0reo
  Resolved https://github.com/NVIDIA/NeMo-Agent-Toolkit.git to commit 5212580317a2abaabb4f80fbb5567dc1c78ef651
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 124.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 111.

In [ ]:
# import logging

# from pydantic import Field

# from aiq.builder.builder import Builder
# from aiq.builder.function_info import FunctionInfo
# from aiq.cli.register_workflow import register_function
# from aiq.data_models.function import FunctionBaseConfig

# logger = logging.getLogger(__name__)


# class MyAgentWorkflow1FunctionConfig(FunctionBaseConfig, name="my_agent_workflow"):
#     """
#     AIQ Toolkit function template. Please update the description.
#     """
#     # Add your custom configuration parameters here


# @register_function(config_type=MyAgentWorkflow1FunctionConfig)
# async def my_agent_workflow1_function(
#     config: MyAgentWorkflow1FunctionConfig, builder: Builder
# ):
#     import os
#     from langchain import hub
#     from langchain.agents import AgentExecutor
#     from langchain.agents import create_react_agent
#     from langchain_nvidia_ai_endpoints import ChatNVIDIA
#     from langchain_community.tools.tavily_search import TavilySearchResults

#     # Initialize a tool to search the web
#     tavily_kwargs = {"max_results": 2, "api_key": os.getenv("TAVILY_API_KEY")}
#     search = TavilySearchResults(**tavily_kwargs)

#     # Create a list of tools for the agent
#     tools = [search]

#     # Initialize a LLM client
#     llm_kwargs = {
#         "model_name": "meta/llama-3.3-70b-instruct",
#         "temperature": 0.0,
#         "max_tokens": 1024,
#         "api_key": os.getenv("NVIDIA_API_KEY"),
#     }
#     llm = ChatNVIDIA(**llm_kwargs)

#     # Use an open source prompt
#     prompt = hub.pull("hwchase17/react-chat")
#     react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt, stop_sequence=["\nObservation"])

#     # Initialize an agent executor to iterate through reasoning steps
#     agent_executor = AgentExecutor(agent=react_agent,
#                                 tools=tools,
#                                 max_iterations=15,
#                                 handle_parsing_errors=True,
#                                 verbose=True)
#     async def _response_fn(input_message: str) -> str:
#         # Process the input_message and generate output
#         response = agent_executor.invoke({"input": input_message, "chat_history": []})
#         return response["output"]

#     try:
#         yield FunctionInfo.create(single_fn=_response_fn)
#     except GeneratorExit:
#         print("Function exited early!")
#     finally:
#         print("Cleaning up my_agent_workflow1 workflow.")



In [ ]:
import getpass
import os

if "NVIDIA_API_KEY" not in os.environ:
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key

Enter your NVIDIA API key: ··········


In [ ]:
if "TAVILY_API_KEY" not in os.environ:
    tavily_api_key = getpass.getpass("Enter your Tavily API key: ")
    os.environ["TAVILY_API_KEY"] = tavily_api_key

Enter your Tavily API key: ··········


In [ ]:
%%bash
mkdir -p notebooks/examples/
cat << EOF > notebooks/examples/langchain_agent.py
import os
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_community.tools.tavily_search import TavilySearchResults

# Initialize a tool to search the web
tavily_kwargs = {"max_results": 2, "api_key": os.getenv("TAVILY_API_KEY")}
search = TavilySearchResults(**tavily_kwargs)

# Create a list of tools for the agent
tools = [search]

# Initialize a LLM client
llm_kwargs = {
    "model_name": "meta/llama-3.3-70b-instruct",
    "temperature": 0.0,
    "max_tokens": 1024,
    "api_key": os.getenv("NVIDIA_API_KEY"),
}
llm = ChatNVIDIA(**llm_kwargs)

# Use an open source prompt
prompt = hub.pull("hwchase17/react-chat")

# Initialize a ReAct agent
react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt, stop_sequence=["\nObservation"])

# Initialize an agent executor to iterate through reasoning steps
agent_executor = AgentExecutor(agent=react_agent,
                               tools=tools,
                               max_iterations=15,
                               handle_parsing_errors=True,
                               verbose=True)

# Invoke the agent with a user query
response = agent_executor.invoke({"input": "Who is the current Pope?", "chat_history": []})

# Print the response
print(response["output"])
EOF

In [ ]:
!python notebooks/examples/langchain_agent.py

/content/notebooks/examples/langchain_agent.py:10: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(**tavily_kwargs)
/content/notebooks/examples/langchain_agent.py:22: DeprecationWarning: The 'max_tokens' parameter is deprecated and will be removed in a future version. Please use 'max_completion_tokens' instead.
  llm = ChatNVIDIA(**llm_kwargs)
/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: "Who is the curr

In [ ]:
!aiq workflow create --workflow-dir notebooks/examples my_agent_workflow

Installing workflow 'my_agent_workflow'...
Workflow 'my_agent_workflow' installed successfully.
Workflow 'my_agent_workflow' created successfully in '/content/notebooks/examples/my_agent_workflow'.


In [ ]:
!aiq run --config_file notebooks/examples/my_agent_workflow/configs/config.yml --input "Who is the current Pope?"

2025-08-01 06:38:03,616 - aiq.runtime.loader - WARNING - Loading module 'aiq.agent.register' from entry point 'aiq_agents' took a long time (618.634701 ms). Ensure all imports are inside your registered functions.
2025-08-01 06:38:03,774 - aiq.cli.commands.start - INFO - Starting AIQ Toolkit from config file: 'notebooks/examples/my_agent_workflow/configs/config.yml'
/content/notebooks/examples/my_agent_workflow/src/my_agent_workflow/my_agent_workflow_function.py:33: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(**tavily_kwargs)
/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided wh

In [ ]:
!aiq info components -t function -q tavily_internet_search

2025-08-01 06:39:10,087 - aiq.runtime.loader - WARNING - Loading module 'aiq.agent.register' from entry point 'aiq_agents' took a long time (606.028080 ms). Ensure all imports are inside your registered functions.
                           AIQ Toolkit Search Results                           
┏━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ package    ┃ version ┃ component_type ┃ component_name   ┃ description       ┃
┡━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ aiqtoolkit │ 1.1.0   │ function       │ tavily_internet_ │ Tool that         │
│            │         │                │ search           │ retrieves         │
│            │         │                │                  │ relevant contexts │
│            │         │                │                  │ from web search   │
│            │         │                │                  │ (using Tavily)    │
│            │         │                │                

In [ ]:
%%bash

# Modify the config file
cat <<EOF > notebooks/examples/my_agent_workflow/configs/config_modified.yml
general:
  use_uvloop: true
  logging:
    console:
      _type: console
      level: WARN

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.3-70b-instruct
    temperature: 0.0
    max_tokens: 1024
    api_key: $NVIDIA_API_KEY

functions:
  my_internet_search:
    _type: tavily_internet_search
    max_results: 2
    api_key: $TAVILY_API_KEY

workflow:
  _type: my_agent_workflow
  tool_names:
    - my_internet_search
  llm_name: nim_llm
  max_history: 10
  max_iterations: 15
  description: "A helpful assistant that can search the internet for information"

EOF

In [ ]:
!aiq run --config_file notebooks/examples/my_agent_workflow/configs/config_modified.yml --input "Who is the current Pope?"


2025-08-01 06:39:23,988 - aiq.runtime.loader - WARNING - Loading module 'aiq.agent.register' from entry point 'aiq_agents' took a long time (641.341686 ms). Ensure all imports are inside your registered functions.
2025-08-01 06:39:24,140 - aiq.cli.commands.start - INFO - Starting AIQ Toolkit from config file: 'notebooks/examples/my_agent_workflow/configs/config_modified.yml'
2025-08-01 06:39:24,144 - aiq.cli.commands.start - WARNING - The front end type in the config file (fastapi) does not match the command name (console). Overwriting the config file front end.
/content/notebooks/examples/my_agent_workflow/src/my_agent_workflow/my_agent_workflow_function.py:33: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_t